In [1]:
import os
import numpy
from datetime import datetime, timedelta

In [2]:
MXD = 'MOD'
ws_folder = '/disk1/workspace/20240116'
RAY_infos_npy = os.path.join(ws_folder, 'analysis', MXD+'03_RAY-matched_infos_10km_land.npy')

In [3]:
ray_matched_infos = numpy.load(RAY_infos_npy)

In [4]:
pixel_res = 0.01
ahi_llon = 85.0
ahi_llat = 60.0

MCD_landcover_npy = os.path.join(ws_folder, 'reference', 'MCD12Q1.006.LC.AHI01km.npy')
mcd_landcover = numpy.load(MCD_landcover_npy)

In [5]:
cloudmask_folder = '/disk4/AHI_CloudMask'

def is_clearsky_record(matched_info_item):    
    lon = float(matched_info_item[0][:-1])
    lat = float(matched_info_item[1][:-1])
    mxd_date = datetime.strptime(matched_info_item[2], "%Y%m%d%H%M")
    ahi_date = mxd_date
    if matched_info_item[2][-1:] =='5':
        ahi_date = mxd_date + timedelta(minutes=5)
    ahi_time_str = ahi_date.strftime("%Y%m%d%H%M")
    if ahi_time_str[-4:] == '0240':
        ahi_date_temp = ahi_date - timedelta(minutes=10)
        ahi_time_str = ahi_date_temp.strftime("%Y%m%d%H%M")

    ahi_cm_dat = os.path.join(cloudmask_folder, ahi_time_str[:6], 'AHIcm.v0.'+ahi_time_str+'.dat')
    cloudmask = None
    cloudmask_res = 0.02
    if not os.path.exists(ahi_cm_dat):
        print('no file:', ahi_cm_dat)
        cloudmask = numpy.zeros((6000, 6000))
    else:
        try:
            cloudmask = numpy.fromfile(ahi_cm_dat, dtype=numpy.float32)
            cloudmask[cloudmask < 0.95] = 0.   # cloud
            cloudmask[cloudmask >= 0.95] = 1.
            cloudmask = cloudmask.reshape(6000, 6000)   # 0.02°
        except Exception as e:
            print(e)
            cloudmask = numpy.zeros((6000, 6000))

    cm_lon_idx = int((lon-85.)/cloudmask_res)
    cm_lat_idx = int((60.-lat)/cloudmask_res)
    is_clearsky = int(cloudmask[cm_lat_idx][cm_lon_idx])
    
    return is_clearsky, ahi_time_str

In [6]:
filter_infos = []
ahi_time_list = []
for ray_matched_info in ray_matched_infos:
    # within Malay 45°VZA region
    lon = round(int(float(ray_matched_info[0])*100)/100,2)
    lat = round(int(float(ray_matched_info[1])*100)/100,2)
    if lon > 90 and lon < 110 and lat > -10 and lat < 15:
        # landcover
        lon_idx = int((lon - ahi_llon)/pixel_res)
        lat_idx = int((ahi_llat - lat)/pixel_res)
        landcover_idx = int(mcd_landcover[lat_idx][lon_idx])
        if landcover_idx in [12, 4]:
#             # between May to August
#             month_str = ray_matched_info[2][4:6]
#             if float(month_str) >= 5 and float(month_str) <= 8:
            # is clear-sky
            is_clearsky, ahi_time_str = is_clearsky_record(ray_matched_info)
            if is_clearsky:
                matched_info = ray_matched_info[:]
                matched_info[0] = str(lon)
                matched_info[1] = str(lat)
                filter_infos.append(matched_info)
                ahi_time_list.append(ahi_time_str)

no file: /disk4/AHI_CloudMask/201810/AHIcm.v0.201810270310.dat
no file: /disk4/AHI_CloudMask/201810/AHIcm.v0.201810270310.dat
no file: /disk4/AHI_CloudMask/201810/AHIcm.v0.201810270310.dat
no file: /disk4/AHI_CloudMask/201810/AHIcm.v0.201810270310.dat
no file: /disk4/AHI_CloudMask/201810/AHIcm.v0.201810270310.dat
no file: /disk4/AHI_CloudMask/201810/AHIcm.v0.201810270310.dat
no file: /disk4/AHI_CloudMask/201810/AHIcm.v0.201810270310.dat
no file: /disk4/AHI_CloudMask/201810/AHIcm.v0.201810270310.dat


In [7]:
len(ray_matched_infos)

47220

In [8]:
len(filter_infos)

1322

In [9]:
numpy.save(os.path.join(ws_folder, 'process', 'MOD_RAY_Malay_45_clear.npy'), numpy.array(filter_infos))

In [10]:
ahi_ac_list = []
for filtered_item_idx in range(len(filter_infos)):
    filtered_item = filter_infos[filtered_item_idx]
    lon = float(filtered_item[0])
    lat = float(filtered_item[1])
    ahi_time = ahi_time_list[filtered_item_idx]
    # landcover
    lon_idx = int((lon - ahi_llon)/pixel_res)
    lat_idx = int((ahi_llat - lat)/pixel_res)
    landcover_idx = int(mcd_landcover[lat_idx][lon_idx])
    ahi_ac_info = [str(lon), str(lat), ahi_time, str(landcover_idx)]
    ahi_ac_list.append(ahi_ac_info)

In [11]:
numpy.save(os.path.join(ws_folder, 'process', 'MOD_RAY_Malay_45_clear_ahi_time.npy'), numpy.array(ahi_ac_list))